- -> exp048
- optuna でハイパラ調整

In [37]:
# %load_ext autotime
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
import copy
import json
import os
import sys
import warnings
from pathlib import Path

import wandb
import optuna

import lightgbm as lgb
import numpy as np
import pandas as pd
import torch
from rdkit import rdBase
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GroupKFold, KFold, ShuffleSplit
from tqdm.auto import tqdm

from sklearn.decomposition import TruncatedSVD
from scipy import sparse

rdBase.DisableLog("rdApp.warning")
warnings.filterwarnings("ignore")

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [46]:
pr_number = 1
debug = False

In [40]:
%autoreload 2

src_path = "../"

sys.path.append(src_path)

from src.data import load_data, add_descriptors, add_descriptors_mordred, add_external_data, make_smile_canonical, add_maccs, add_augumented_gmm, add_graph_features, add_count_atoms
from src.model import train_lgb_for_target, save_lgb_model
from src.utils import NULL_FOR_SUBMISSION, generate_scaffold, score, add_scaffold_kfold, scaffold_cv_split, get_useless_cols


In [47]:
exp = "exp049"
notes = "optuna 探索"
model_name = "lgb"

config = {
    "debug": debug,
    "n_splits": 3,
    "num_epochs": 1000,
    "batch_size": 128,
    "drop_ratio": 0.5,
    "force_update_train": False,
    "augumented_gmm": False,
    "is_complement": True,
    "remove_external_cv": True,
}

dataset_title = f"model-{exp}"
dataset_id = f"koya346/{dataset_title}"


if config["debug"]:
    config["num_epochs"] = 10

targets = ["Tg", "FFV", "Tc", "Density", "Rg"]
org_target_cols = [f"org_{target}" for target in targets]        

# TODO: 学習パラメータ定義
params = {
    "objective": "regression",
    "metric": "mae",
    "verbosity": -1,
    "learning_rate": 0.01,
    "max_depth": 7,
    "seed": 42,
    "subsample": 0.7,
    "colsample_bytree": 0.6,
    "num_boost_round": 20000,
}

config.update(params)
pre_params = copy.deepcopy(params)
pre_params["num_boost_round"] = 300


In [58]:
# --------------------------------------------------
# 必須: org-only の fold 評価を返す関数
# --------------------------------------------------
def cv_mae_lgbm(params, X_df, y_series, target_col, target_features, early_stopping_rounds=100):
    """
    params: dict (LightGBM のパラメータ)
    X_df:   学習用の特徴 DataFrame（外部補完を含んでもOK）
    y_series: 目的変数（学習に使う列。外部補完された値も含む想定）
    target_col: 目的変数名（例: 'Tg'）
    target_features: 使用する特徴量列名のリストを要素に持つリスト（前処理済）
    return: (fold_mae_list, fold_best_iterations)
    """

    fold_mae_list = []
    fold_best_iters = []

    # X_df には org の列（例: f"org_{target_col}"）も入っている前提
    # 評価は org-only
    org_target_col = f"org_{target_col}"
    for fold, tr_idx, val_idx in scaffold_cv_split(X_df, target=target_col, n_splits=config["n_splits"], remove_external=config["remove_external_cv"]):
        features = target_features[fold]
        # 学習データ（外部補完を含む）
        X_tr, y_tr = X_df.loc[tr_idx, features], y_series.loc[tr_idx]
        X_va, y_va = X_df.loc[val_idx, features], y_series.loc[val_idx]

        lgb_trn = lgb.Dataset(X_tr, label=y_tr)
        lgb_val = lgb.Dataset(X_va, label=y_va, reference=lgb_trn)

        # 目的：MAE（Huberの場合も metric は l1 を見る）
        run_params = {
            "boosting_type": params.get("boosting", "gbdt"),
            "objective": params.get("objective", "mae"),
            "metric": "l1",
            "learning_rate": params.get("learning_rate", 0.03),
            "num_leaves": params.get("num_leaves", 63),
            "max_depth": params.get("max_depth", -1),
            "min_data_in_leaf": params.get("min_data_in_leaf", 120),
            "feature_fraction": params.get("feature_fraction", 0.75),
            "bagging_fraction": params.get("bagging_fraction", 0.8),
            "bagging_freq": params.get("bagging_freq", 1),
            "lambda_l1": params.get("lambda_l1", 0.0),
            "lambda_l2": params.get("lambda_l2", 0.0),
            "min_split_gain": params.get("min_split_gain", 0.0),
            "max_bin": params.get("max_bin", 255),
            "verbose": -1,
            "seed": params.get("seed", 42),
            "feature_fraction_seed": params.get("feature_fraction_seed", 42),
            "bagging_seed": params.get("bagging_seed", 42),
        }
        if run_params["objective"] == "huber":
            run_params["alpha"] = 0.9  # デフォに近い値
            run_params["huber_delta"] = params.get("huber_delta", 2.0)
        if run_params["boosting_type"] == "dart":
            run_params["drop_rate"] = params.get("drop_rate", 0.1)
            run_params["skip_drop"] = params.get("skip_drop", 0.5)

        model = lgb.train(
            run_params,
            lgb_trn,
            num_boost_round=params.get("num_boost_round", 5000),
            valid_sets=[lgb_trn, lgb_val],
            valid_names=["train", "valid"],
            callbacks=[lgb.early_stopping(early_stopping_rounds, verbose=False)],
        )

        pred = model.predict(X_va, num_iteration=model.best_iteration)
        fold_mae = mean_absolute_error(y_va, pred)

        fold_mae_list.append(fold_mae)
        fold_best_iters.append(model.best_iteration)

    return fold_mae_list, fold_best_iters

# --------------------------------------------------
# Optuna の objective
# --------------------------------------------------
def objective_lgbm(trial, X_df, y_series, target_col, target_features):
    # 探索空間
    boosting = trial.suggest_categorical("boosting", ["gbdt", "dart"])
    objective = trial.suggest_categorical("objective", ["mae", "huber"])

    params = {
        "boosting": boosting,
        "objective": objective,
        "learning_rate": trial.suggest_float("learning_rate", 0.02, 0.05),
        "num_leaves": trial.suggest_categorical("num_leaves", [31, 63, 127, 255]),
        "max_depth": trial.suggest_categorical("max_depth", [-1, 8, 10, 12]),
        "min_data_in_leaf": trial.suggest_categorical("min_data_in_leaf", [50, 100, 150, 200]),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 0.9),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.7, 0.9),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1, 2]),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-3, 1.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-3, 1.0, log=True),
        "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 0.1),
        "max_bin": 255,
        "num_boost_round": 5000,
        "seed": 42,
        "feature_fraction_seed": trial.suggest_int("feature_fraction_seed", 1, 9999),
        "bagging_seed": trial.suggest_int("bagging_seed", 1, 9999),
    }
    if boosting == "dart":
        params["drop_rate"] = trial.suggest_float("drop_rate", 0.05, 0.2)
        params["skip_drop"] = trial.suggest_float("skip_drop", 0.2, 0.8)
    if objective == "huber":
        params["huber_delta"] = trial.suggest_float("huber_delta", 1.0, 5.0)

    # CV 実行（org-only評価）
    fold_mae, fold_best = cv_mae_lgbm(params, X_df, y_series, target_col, target_features)

    # 途中経過を記録（プルーニングしたければここで trial.report/should_prune）
    if len(fold_mae) > 0:
        trial.report(np.mean(fold_mae), step=0)
        # if trial.should_prune():
        #     raise optuna.TrialPruned()

    # 目的は MAE を最小化
    score = np.mean(fold_mae) if len(fold_mae) else 1e9
    # best_iteration の中央値を属性として持たせておく
    trial.set_user_attr("best_iterations", fold_best)

    return score

def tune_lgbm_for_target(X_df, y_series, target_col, target_features, n_trials=500):
    """
    X_df:  特徴＋ fold + org_{target} を含む学習テーブル
    y_series: 学習に使う目的列（= 外部補完含む値）
    target_col: 目的名（例: 'Tg'）
    target_features: 使用列リストを要素とするリスト
    """
    study = optuna.create_study(direction="minimize", study_name=f"lgbm_{target_col}")
    study.optimize(lambda tr: objective_lgbm(tr, X_df, y_series, target_col, target_features),
                   n_trials=n_trials, gc_after_trial=True)

    best_params = study.best_trial.params
    best_iters = study.best_trial.user_attrs.get("best_iterations", [])
    best_iter_med = int(np.median(best_iters)) if len(best_iters) else 1000

    print(f"[{target_col}] best MAE (OOF): {study.best_value:.5f}")
    print(f"[{target_col}] best params:", best_params)
    print(f"[{target_col}] median(best_iteration across folds) = {best_iter_med}")

    return best_params, best_iter_med, study

In [59]:
wandb_name = f"{exp}_{model_name}" if not config["debug"] else f"{exp}_{model_name}_debug"
wandb.init(project="opp2025", name=wandb_name, config=config)
wandb.log({"Notes": notes})

# ---------------------------
# メイン処理
# ---------------------------
if config["debug"]:
    output_path = Path("/home/kouya-takahashi/kaggle/opp2025/outputs") / exp / "debug"
else:
    output_path = Path("/home/kouya-takahashi/kaggle/opp2025/outputs") / exp

model_output_path = output_path / "model_cv"
if not os.path.exists(model_output_path):
    os.makedirs(model_output_path)

data_dir = Path("/home/kouya-takahashi/kaggle/opp2025/data/raw")

# 学習データ用意

if os.path.exists(output_path / "train.csv") and not config["force_update_train"]:
    train = pd.read_csv(output_path / "train.csv")
else:
    train, _ = load_data(data_dir)
    # 外部データ判定用に元の目的変数を保持しておく
    # 外部データ判定は scaffold_cv_split 内で行う
    for target in targets:
        train[f"org_{target}"] = train[target]
    

    train["org_SMILES"] = train["SMILES"]
    train["SMILES"] = train["SMILES"].apply(make_smile_canonical)
    if config["debug"]:
        # 各ターゲットが欠損していないデータを30 件ずつ取り出す
        tmp_dfs = []
        for target in targets:
            cond = train[target].notnull()
            tmp_dfs.append(train[cond].iloc[:30])
        train = pd.concat(tmp_dfs).reset_index(drop=True)
    else:
        print(train.shape)
        external_data_dict = [
            {
                "ex_path": data_dir / "neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv",
                "col": "Tg",
            },
            {
                "ex_path": data_dir / "neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv",
                "col": "Tc",
                "rename_d": {"TC_mean": "Tc"},
            },
            {
                "ex_path": data_dir / "neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv",
                "col": "FFV",
            },
            {
                "ex_path": data_dir / "tg-smiles-pid-polymer-class/TgSS_enriched_cleaned.csv",
                "col": "Tg",
            },
            {
                "ex_path": data_dir / "smiles-extra-data/data_dnst1.xlsx",
                "col": "Density",
                "rename_d": {"density(g/cm3)": "Density"}, 
            },
            {
                "ex_path": data_dir / "smiles-extra-data/data_tg3.xlsx",
                "col": "Tg",
                "rename_d": {"Tg [K]": "Tg"}, 
            },
            {
                "ex_path": data_dir / "smiles-extra-data/JCIM_sup_bigsmiles.csv",
                "col": "Tg",
                "rename_d": {"Tg (C)": "Tg"}, 
            },
        ]
        for d in external_data_dict:
            print(f"ex_path: {str(d['ex_path'])}")
            train = add_external_data(
                df=train,
                ex_path=d.get("ex_path"),
                col=d.get("col"),
                rename_d=d.get("rename_d"),
                is_complement=config["is_complement"]
            )
            print("after train.shape: ", train.shape)

    train["is_external"] = train["id"].isnull()
    train = add_maccs(train)

    # rdkit の記述子, morgan finger print
    train = add_descriptors(train, radius=2, fp_size=1024)
    train = add_descriptors_mordred(train, num_confs=10, ignore_3D=False)

    new_cols = []
    seen = {}
    for col in train.columns:
        if col in seen:
            seen[col] += 1
            new_cols.append(f"{col}_{seen[col]}")
        else:
            seen[col] = 0
            new_cols.append(col)
    
    train.columns = new_cols
    
    # グラフ特徴量
    train = add_graph_features(train)
    train = add_count_atoms(train)
    
    train["id"] = np.arange(len(train))
    features = train.drop(targets + org_target_cols + ["id", "org_SMILES", "SMILES", "is_external"], axis=1).columns
    for col in features:
        if train[col].dtype == "object":
            train[col] = pd.to_numeric(train[col], errors="coerce")
    useless_cols = get_useless_cols(train.drop(targets + org_target_cols + ["id", "org_SMILES", "SMILES", "is_external"], axis=1))
    
    train = train.drop(useless_cols, axis=1)
    
    os.makedirs(output_path, exist_ok=True)
    train.to_csv(output_path / "train.csv", index=False)
    print("Saved train.csv")

# 不要なビット列を除去
bit_cols = []
remove_cols = []

for col in train.drop(targets + org_target_cols + ["id", "org_SMILES", "SMILES", "is_external"], axis=1).columns:
    if len(train[col].unique()) != 2:
        continue
    if np.all(train[col].unique() == np.array([0, 1])):
        bit_cols.append(col)
        p = train[col].mean()
        if p > 0.01 and p < 0.99:
            continue
        remove_cols.append(col)
print("bit cols: ", len(bit_cols))
print("remove cols: ", len(remove_cols))

train = train.drop(remove_cols, axis=1)

# 計算済の fold を突合
folds = pd.read_csv("/home/kouya-takahashi/kaggle/opp2025/data/preprocess/fold/folds.csv")
train = train.merge(folds[["SMILES", "fold"]], how="left", on="SMILES")

features = train.drop(targets + org_target_cols + ["id", "org_SMILES", "SMILES", "fold", "is_external"], axis=1).columns
print(len(features))
oof_dfs = []

loss_table_wandb = wandb.Table(["exp", "model_name", "fold", "target", "mae", "mse"])
all_loss_tables = []
mae_dict = {}
all_models = {}
all_features = {}

# 特徴量選定
for idx, target_col in enumerate(targets):
    df_train = train[train[target_col].notnull()].reset_index(drop=True)
    X = df_train[features]
    y = df_train[target_col]
    features_by_folds = []
    for fold, tr_idx, val_idx in scaffold_cv_split(df_train, target=target_col, n_splits=config["n_splits"], remove_external=config["remove_external_cv"]):
        if len(tr_idx) == 0 or len(val_idx) == 0:
            print(f"Skip fold... tr_idx: {len(tr_idx)}, val_idx: {len(val_idx)}")
            continue
        hits = np.zeros(X.shape[1], dtype=int)

        rs = ShuffleSplit(n_splits=3, train_size=0.9, random_state=42)
        for idx, (tr_in, va_in) in enumerate(rs.split(tr_idx)):
            X_tr_in, X_va_in = X.iloc[tr_in][features], X.iloc[va_in][features]
            y_tr_in, y_va_in = y.iloc[tr_in], y.iloc[va_in]
            dtrain_in = lgb.Dataset(X_tr_in, label=y_tr_in)
            dvalid_in = lgb.Dataset(X_va_in, label=y_va_in)
            model = lgb.train(
                params,
                dtrain_in,
                valid_sets=[dtrain_in, dvalid_in],
                callbacks=[
                    lgb.early_stopping(stopping_rounds=50),
                    lgb.log_evaluation(200)
                ]
            )
            rank = model.feature_importance("gain").argsort()[::-1]
            topk = set(rank[:int(0.3 * len(rank))])
            hits[list(topk)] += 1
        keep = hits >= 2
        use_features = features[keep]
        features_by_folds.append(use_features)
    all_features[target_col] = features_by_folds

# ハイパラ調整とスコア確認
for idx, target_col in enumerate(targets):
    target_features = all_features[target_col]
    loss_tables = []
    print(f"\n=== Training for target: {target_col} ===")
    
    df_train = train[train[target_col].notnull()].reset_index(drop=True)
    y = df_train[target_col]
    oof = np.full(len(df_train), np.nan, dtype=float)
    
    models = []
    print("n features: ", [len(feat) for feat in target_features])
    best_params, best_iter_med, study = tune_lgbm_for_target(df_train, y, target_col, target_features, n_trials=60)

    for fold, tr_idx, val_idx in scaffold_cv_split(df_train, target=target_col, n_splits=config["n_splits"], remove_external=config["remove_external_cv"]):
        features = target_features[fold]
        
        if len(tr_idx) == 0 or len(val_idx) == 0:
            print(f"Skip fold... tr_idx: {len(tr_idx)}, val_idx: {len(val_idx)}")
            continue
        loss_table = {}
        print(f"fold: {fold + 1}")
        use_features = features
        print(len(use_features))        

        # 特徴量選択して valid データとともに学習
        X_train, X_val = df_train.iloc[tr_idx][use_features], df_train.iloc[val_idx][use_features]
        y_train, y_val = y.iloc[tr_idx], y.iloc[val_idx]

        if config["augumented_gmm"]:
            X_train, y_train = add_augumented_gmm(X_train, y_train)    
        
        dtrain = lgb.Dataset(X_train, label=y_train)
        dval = lgb.Dataset(X_val, label=y_val, reference=dtrain)

        model = lgb.train(
            best_params,
            dtrain,
            valid_sets=[dtrain, dval],
            callbacks=[
                lgb.early_stopping(stopping_rounds=50),
                lgb.log_evaluation(200)
            ]
        )

        save_lgb_model(model, str(model_output_path / f"model_{target_col}_{fold}.txt"))

        pred = model.predict(X_val, num_iteration=model.best_iteration)
        oof[val_idx] = pred

        mse = mean_squared_error(y_val, pred)
        mae = mean_absolute_error(y_val, pred)
        print(f"fold: {fold}, target: {target_col}, mae: {mae}")
        loss_table["fold"] = fold
        loss_table["target"] = target_col
        loss_table["mae"] = mae
        loss_table["mse"] = mse

        loss_tables.append(loss_table)
        models.append(model)

    # 評価に使うのは元データのみ
    cond = (~np.isnan(oof)) & (df_train[f"org_{target_col}"].notnull())
    y_true = y[cond]
    y_pred = oof[cond]
    score_mse = mean_squared_error(y_true, y_pred)
    score_mae = mean_absolute_error(y_true, y_pred)
    print(f"RMSE for {target_col}: {score_mse:.4f}")
    print(f"MAE for {target_col}: {score_mae:.4f}")
    mae_dict[target_col] = score_mae

    for loss_table in loss_tables:
        loss_table_wandb.add_data(exp, model_name, loss_table["fold"], loss_table["target"], loss_table["mae"], loss_table["mse"])
    all_loss_tables += loss_tables

    oof_df = pd.DataFrame({
        "id": df_train["id"].values,
        target_col: oof
    })
    oof_dfs.append(oof_df)   

    all_models[target_col] = models

wandb.log({"fold_target_losses": loss_table_wandb})
# target 毎の 平均 mae を記録
for target in targets:
    key_name = f"{target}_mean_mae"
    mae_values = mae_dict[target]
    # mae_values = [d["mae"] for d in all_loss_tables if d["target"] == target]
    wandb.log({key_name: np.mean(mae_values)})

# CV 計算
cond = ~train["is_external"]
oof_df = pd.DataFrame()

# 元の目的変数も入れておく
for target in targets:
    oof_df[f"org_{target}"] = train.loc[cond, f"org_{target}"]

oof_df["id"] = train.loc[cond, "id"]

# 正規化前の SMILES
oof_df["SMILES"] = train.loc[cond, "org_SMILES"]

for i_oof in oof_dfs:
    oof_df = oof_df.merge(i_oof, on="id", how="left")

# # 予測しなかった部分は null にしておく
# for target in targets:
#     oof_df.loc[oof_df[target] == 0, target] = np.nan

oof_df.to_csv(output_path / "oof.csv", index=False)

solution = train.loc[cond, ["id"] + org_target_cols].copy()
solution.columns = ["id"] + targets

# solution = solution.fillna(NULL_FOR_SUBMISSION)

# oof_df = oof_df.fillna(NULL_FOR_SUBMISSION)

# 評価
final_score = score(
    solution=solution,
    submission=oof_df,
)
print(f"\n📊 Final OOF Score (wMAE): {final_score:.6f}")
wandb.log({"wMAE": final_score})

# target 毎の best_iteration を保存する。保存したモデルには記録されてなかった
best_iterations = {}
for target in targets:
    target_best_iterations = [model.best_iteration for model in all_models[target]]
    best_iterations[target] = target_best_iterations
print(best_iterations)

with open(output_path / "best_iterations.json", "w") as f:
    json.dump(best_iterations, f)

Notes,optuna 探索


bit cols:  480
remove cols:  47
581
train rows: 24, valid rows: 12, ignore rows: 0
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	training's l1: 61.2778	valid_1's l1: 103.248
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	training's l1: 71.4382	valid_1's l1: 58.7813
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	training's l1: 69.4325	valid_1's l1: 54.2674
train rows: 22, valid rows: 14, ignore rows: 0
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	training's l1: 62.7842	valid_1's l1: 112.407
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	training's l1: 68.0968	valid_1's l1: 94.3228
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	training's l1: 77.4697	valid_1's l1: 37.4573


[I 2025-08-30 13:27:43,292] A new study created in memory with name: lgbm_Tg
[I 2025-08-30 13:27:43,320] Trial 0 finished with value: 56.401018078755804 and parameters: {'boosting': 'gbdt', 'objective': 'mae', 'learning_rate': 0.03489328487055608, 'num_leaves': 31, 'max_depth': -1, 'min_data_in_leaf': 50, 'feature_fraction': 0.88882487416645, 'bagging_fraction': 0.7483773947529123, 'bagging_freq': 2, 'lambda_l1': 0.003238729512274587, 'lambda_l2': 0.044908918376439434, 'min_split_gain': 0.06369468611196646, 'feature_fraction_seed': 7449, 'bagging_seed': 1138}. Best is trial 0 with value: 56.401018078755804.


[200]	training's l1: 3.0714	valid_1's l1: 3.32401
[400]	training's l1: 2.33358	valid_1's l1: 2.73981
Early stopping, best iteration is:
[477]	training's l1: 2.15441	valid_1's l1: 2.65707
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	training's l1: 5.03009	valid_1's l1: 5.55013
Training until validation scores don't improve for 50 rounds
[200]	training's l1: 3.03086	valid_1's l1: 3.43994
[400]	training's l1: 2.29296	valid_1's l1: 2.88789
Early stopping, best iteration is:
[459]	training's l1: 2.17478	valid_1's l1: 2.84422

=== Training for target: Tg ===
n features:  [174, 174, 174]
train rows: 24, valid rows: 12, ignore rows: 0
train rows: 22, valid rows: 14, ignore rows: 0
train rows: 26, valid rows: 10, ignore rows: 0
train rows: 24, valid rows: 12, ignore rows: 0

[I 2025-08-30 13:27:43,490] Trial 1 finished with value: 61.223520324540345 and parameters: {'boosting': 'gbdt', 'objective': 'huber', 'learning_rate': 0.04434629708340425, 'num_leaves': 255, 'max_depth': 8, 'min_data_in_leaf': 100, 'feature_fraction': 0.6625030891714245, 'bagging_fraction': 0.761863370318359, 'bagging_freq': 1, 'lambda_l1': 0.015328341556948061, 'lambda_l2': 0.38458539701405997, 'min_split_gain': 0.03590948524898107, 'feature_fraction_seed': 1488, 'bagging_seed': 6183, 'huber_delta': 3.726207518776876}. Best is trial 0 with value: 56.401018078755804.
[I 2025-08-30 13:27:43,655] Trial 2 finished with value: 56.401018078755804 and parameters: {'boosting': 'gbdt', 'objective': 'mae', 'learning_rate': 0.02431548277860637, 'num_leaves': 63, 'max_depth': 10, 'min_data_in_leaf': 50, 'feature_fraction': 0.6604371042140155, 'bagging_fraction': 0.8684830274417583, 'bagging_freq': 2, 'lambda_l1': 0.003155808957537449, 'lambda_l2': 0.0018748754692317646, 'min_split_gain': 0.02167

...

train rows: 26, valid rows: 10, ignore rows: 0
train rows: 24, valid rows: 12, ignore rows: 0
train rows: 22, valid rows: 14, ignore rows: 0


[I 2025-08-30 13:27:57,429] Trial 54 finished with value: 56.401018078755804 and parameters: {'boosting': 'dart', 'objective': 'mae', 'learning_rate': 0.02699185874859829, 'num_leaves': 127, 'max_depth': 8, 'min_data_in_leaf': 50, 'feature_fraction': 0.8963994017259376, 'bagging_fraction': 0.7367692869690866, 'bagging_freq': 1, 'lambda_l1': 0.33612749639044776, 'lambda_l2': 0.0028914267553496477, 'min_split_gain': 0.016470862133522522, 'feature_fraction_seed': 784, 'bagging_seed': 2212, 'drop_rate': 0.07450851461694824, 'skip_drop': 0.5870269517913339}. Best is trial 0 with value: 56.401018078755804.


train rows: 26, valid rows: 10, ignore rows: 0


[W 2025-08-30 13:27:57,571] Trial 55 failed with parameters: {'boosting': 'gbdt', 'objective': 'mae'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/kouya-takahashi/kaggle/opp2025/.venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_877964/1556373603.py", line 131, in <lambda>
    study.optimize(lambda tr: objective_lgbm(tr, X_df, y_series, target_col, target_features),
  File "/tmp/ipykernel_877964/1556373603.py", line 85, in objective_lgbm
    "learning_rate": trial.suggest_float("learning_rate", 0.02, 0.05),
  File "/home/kouya-takahashi/kaggle/opp2025/.venv/lib/python3.10/site-packages/optuna/trial/_trial.py", line 161, in suggest_float
    suggested_value = self._suggest(name, distribution)
  File "/home/kouya-takahashi/kaggle/opp2025/.venv/lib/python3.10/site-packages/optuna/trial/_trial.py", line 635, in _suggest
    param_value = self

KeyboardInterrupt: 